# Autograd: 자동 미분

autograd 패키지는 Tensor의 모든 연산에 대해 자동 미분을 제공합니다. 이는 실행-기반-정의(define-by-run) 프레임워크로, 이는 코드를 어떻게 작성하여 실행하느냐에 따라 역전파가 정의된다는 뜻이며, 역전파는 학습 과정의 매 단계마다 달라집니다.

## 변수 Variable

패키지의 중심에는 autograd.Variable 클래스가 있습니다. 이는 Tensor를 감싸고(wrap) 있으며, Tensor 기반으로 정의된 거의 대부분의 연산을 지원합니다. 계산이 완료된 후 .backward() 를 호출하여 모든 변화도(gradient)을 자동으로 계산할 수 있습니다.  

.data 속성을 사용하여 tensor 자체(raw tensor)에 접근할 수 있으며, 이 변수와 관련된 변화도는 .grad 에 누적됩니다.  

Autograd 구현에서 매우 중요한 클래스가 하나 더 있는데요, 바로 Function 클래스입니다.  

Variable 과 Function 은 상호 연결되어 있으며, 모든 연산 과정을 부호화(encode)하여 순환하지 않은 그래프(acyclic graph)를 생성합니다. 각 변수는 .grad_fn 속성을 갖고 있는데, 이는 Variable 을 생성한 Function 을 참조하고 있습니다. (단, 사용자가 만든 Variable은 예외로, 이 때 grad_fn 은 None 입니다.)  

도함수를 계산하기 위해서는, Variable 의 .backward() 를 호출하면 됩니다. Variable 이 스칼라(scalar)인 경우(예. 하나의 요소만 갖는 등)에는, backward 에 인자를 정해줄 필요가 없습니다. 하지만 여러 개의 요소를 갖고 있을 때는 tensor의 모양을 gradient 의 인자로 지정할 필요가 있습니다.  

In [30]:
import torch
from torch.autograd import Variable

In [31]:
# Create a variable
x = Variable(torch.ones(2, 2), requires_grad=True)
print(x)

tensor([[1., 1.],
        [1., 1.]], requires_grad=True)


In [32]:
y = x + 2
print(y)

tensor([[3., 3.],
        [3., 3.]], grad_fn=<AddBackward0>)


여기서 y는 연산의 결과로 생성된 것으므로, grad_fn을 갖는다.

In [33]:
print(y.grad_fn)

In [34]:
z = y * y * 3
out = z.mean()

print(z, out)

tensor([[27., 27.],
        [27., 27.]], grad_fn=<MulBackward0>) tensor(27., grad_fn=<MeanBackward0>)


## 변화도 Gradient

In [35]:
out.backward()

In [36]:
print(x.grad)  
# d(out)/dx 인데, out = 3 * (x + 2) * (x + 2) * 3 이므로, 3/2 * (x + 2) 이다. 
# x = 1이므로, 3/2 * 3 = 4.5가 나와야 한다. 

tensor([[4.5000, 4.5000],
        [4.5000, 4.5000]])


In [37]:
x = torch.randn(3)
x = Variable(x, requires_grad=True)

y = x * 2
# norm()은 벡터의 크기를 반환한다.
# 즉, y의 L2 norm이 1000보다 작은 동안同案, y를 2배씩 계속 곱한다.
while y.data.norm() < 1000:  
  y = y * 2

print(y)

tensor([-931.8143, 1383.0795,   31.5079], grad_fn=<MulBackward0>)


In [47]:
# L2 norm test
x = torch.tensor([1.0, 1.0, 1.0])
print(x.data.norm())
a = torch.arange(9, dtype= torch.float) - 4
b = a.reshape((3, 3))
print(a)
print(b)
print(a.data.norm())
print(b.data.norm())

tensor(1.7321)
tensor([-4., -3., -2., -1.,  0.,  1.,  2.,  3.,  4.])
tensor([[-4., -3., -2.],
        [-1.,  0.,  1.],
        [ 2.,  3.,  4.]])
tensor(7.7460)
tensor(7.7460)


In [49]:
x = torch.tensor([1.0, 1.0])
print(x.data.norm())  # 루트 2
x = torch.tensor([1.0, 1.0, 1.0])
print(x.data.norm())  # 루트 3

tensor(1.4142)
tensor(1.7321)


pytorch에서의 norm()은 기본적으로 L2 norm로 계산되며, 이는 차원에서의 기하학적 거리이다.

In [38]:
gradient = torch.FloatTensor([0.1, 1.0, 0.0001])
y.backward(gradient)

print(x.grad)

tensor([2.0480e+02, 2.0480e+03, 2.0480e-01])
